#### Learned Bloom Filter

A learned bloom filter uses a `learned oracle` as a pre-filtering stage in front of a standard bloom filter, called the back-up bloom filter. The oracle function $f(x)$ represents $Pr[x \in S]$. We set a threshold $\tau$ such that if $f(x) \geq \tau$, we return `True` for the query. Otherwise, if $f(x) < \tau$, we send the query to the back-up bloom filter.

Given a set of elements $S$ from universe $U$, we can train the oracle using any ML model on the binary classification task with negative log likelihood loss function. The training dataset contains `(query, label)` pairs, with both positive and negative samples. 

Positive samples are all the elements from $S$ and negative samples are all the elements $U\setminus S$.

We keep a held-out set of negative samples $Q$ with which we tune the threshold $\tau$ to acheive a desired false positive rate $\delta$.

The goal is to create a 2-stage bloom filter that consumes less space than a standard bloom filter while achieving the same (or better) false positive rate.
